In [ ]:
# we will predict housing prices in Boston sub-urbs in the mid 1970s based on data points (crime, tex, etc.)

import keras
from keras.datasets import boston_housing
from keras import layers

import numpy as np
import matplotlib.pyplot as plt


(train_data, train_targets), (test_data, test_targets) = (
    boston_housing.load_data())

# transform data, we don't like the fact that the data points all fall within different ranges 
mean = train_data.mean(axis=0)
train_data -= mean
std = train_data.std(axis=0)
train_data /= std
test_data -= mean
test_data /= std

def build_model():
  model = keras.Sequential([
      layers.Dense(64, activation="relu"),
      layers.Dense(64, activation="relu"),
      layers.Dense(1)
  ])

  model.compile(optimizer="rmsprop", loss="mse", metrics=["mae"])
  return model

# we. will use 4-fold cross-validation for this task 
k=4
num_val_samples = len(train_data) // k 
num_epochs = 130
all_mae_histories = []
all_scores = []

for i in range(k):
  print(f"Processing fold #{i}")
  val_data = train_data[i * num_val_samples: (i + 1) * num_val_samples] 
  val_targets = train_targets[i * num_val_samples: (i + 1) * num_val_samples] 
  
  partial_train_data = np.concatenate(
    [train_data[:i * num_val_samples], 
     train_data[(i + 1) * num_val_samples:]],
    axis=0)
  
  partial_train_targets = np.concatenate(
      [train_targets[:i * num_val_samples], 
       train_targets[(i + 1) * num_val_samples:]],
      axis=0)
  
  model = build_model()

  history = model.fit(partial_train_data, partial_train_targets,
                      validation_data=(val_data, val_targets),
                      epochs=num_epochs, batch_size=16, verbose=0)
    
  mae_history = history.history["val_mae"]
  all_mae_histories.append(mae_history)
  
  val_mse, val_mae = model.evaluate(val_data, val_targets, verbose=0)
  all_scores.append(val_mae)

print(all_scores)
print(np.mean(all_scores))

plt.plot(range(1, len(average_mae_history) + 1), average_mae_history)
plt.xlabel("Epochs")
plt.ylabel("Validation MAE")
plt.show()





